In [3]:
import pandas as pd 
df=pd.read_csv("weather.csv") 
def entropy(probs):
    import math 
    return sum([-prob*math.log(prob,2) for prob in probs]) 

def entropy_of_list(a_list): 
    from collections import Counter 
    cnt=Counter(x for x in a_list) 
    num_instances=len(a_list) 
    probs=[x/num_instances for x in cnt.values()] 
    return entropy(probs) 

total=entropy_of_list(df['play']) 
print("\n Total entropy of play tennis dataset:",total) 

def information_gain(df,split_attr,tattr,trace=0): 
    df_split=df.groupby(split_attr) 
    for name,group in df_split: 
        nobs=len(df.index) 
 
    df_agg=df_split.agg({tattr:[entropy_of_list,lambda x:len(x)/nobs]})[tattr] 
    df_agg.columns=['Entropy','Propobservations'] 
    if trace: 
        print(df_agg) 
    new=sum(df_agg['Entropy']*df_agg['Propobservations']) 
    old=entropy_of_list(df[tattr]) 
    return old-new 

print('info-gain for Outlook is :'+str(information_gain(df,'outlook','play')),"\n") 
print('Info-gain for Humidity is:'+str(information_gain(df,'humidity', 'play')),"\n") 
print('Info-gain for Wind is:'+str(information_gain(df,'wind','play')),"\n") 
print('Info-gain for Temperature is:'+str(information_gain(df,'temperature', 'play')),"\n") 

def id3(df,tattr,attr,default=None): 
    from collections import Counter 
    cnt=Counter(x for x in df[tattr]) 
    if len(cnt)==1: 
        return next(iter(cnt)) 
    elif df.empty or (not attr): 
        return default 
    else: 
        default=max(cnt.keys())
        
    gainz=[information_gain(df,attr,tattr) for attr in attr] 
    index_of_max=gainz.index(max(gainz)) # Index of Best Attribute 
    best_attr=attr[index_of_max] 
    tree={best_attr:{}} # Initiate the tree with best attribute as a node 
    remaining_attr=[i for i in attr if i!=best_attr] 
    for attr_val,data_subset in df.groupby(best_attr): 
        subtree=id3(data_subset,tattr,remaining_attr,default) 
        tree[best_attr][attr_val] =subtree 
    return tree 
 
attr=list(df.columns) 
attr.remove('play') #Remove the class attribute 
from pprint import pprint #pprint means pretty print 
tree=id3(df,'play',attr) 
print("\n\nThe Resultant Decision Tree is:\n") 
pprint(tree) 
 
def predict(query,tree,default=1): 
    for key in list(query.keys()): 
        if key in list(tree.keys()): 
            try: 
                result = tree[key][query[key]] 
            except: 
                return default 
            result = tree[key][query[key]] 
            if isinstance(result,dict): 
                return predict(query,result) 
            else: 
                return result 
query={'outlook':'rain','temperature':'cool','humidity':'normal','wind':'weak'} 
answer=predict(query,tree) 
print("Sample:") 
print(query) 
print('\nCan tennis be played for the given sample: '+answer)


 Total entropy of play tennis dataset: 0.9402859586706309
info-gain for Outlook is :0.2467498197744391 

Info-gain for Humidity is:0.15183550136234136 

Info-gain for Wind is:0.04812703040826927 

Info-gain for Temperature is:0.029222565658954647 



The Resultant Decision Tree is:

{'outlook': {'overcast': 'yes',
             'rain': {'wind': {'strong': 'no', 'weak': 'yes'}},
             'sunny': {'humidity': {'high': 'no', 'normal': 'yes'}}}}
Sample:
{'outlook': 'rain', 'temperature': 'cool', 'humidity': 'normal', 'wind': 'weak'}

Can tennis be played for the given sample: yes
